In [1]:
%load_ext notexbook

%texify

# Compare Collections

In this notebook we will compare the MTG Manager data for collections gathered from the two versions of the App, namely _MTGManager_, and _MTGDragonShield_. 

The data considered in the comparison for each APP is the `"updated"` version, resulting from the whole set of operations applied to align the two different _data layouts_. A thorough description of these operations is reported in the [Collection Analysis](./Collection_Analysis.ipynb) notebook.

**Original** (as in _unmodified_) exported data will be also used accordingly as for reference, and double-check.

In [2]:
import os
from pathlib import Path

from mtg import Collection, ScryfallDB

In [3]:
from mtg import DATA_FOLDER

MTG_MANAGER = DATA_FOLDER / "MTGManager" / "updated"
MTG_DRAGONSHIELD = DATA_FOLDER / "MTGDragonShield" / "updated"

In [4]:
from functools import partial 

def fullpath(path: str, base: Path) -> Path:
    return base / path

manager_path = partial(fullpath, base=MTG_MANAGER)
dshield_path = partial(fullpath, base=MTG_DRAGONSHIELD)

manager_files = list(map(manager_path, filter(lambda f: f.endswith(".csv"), os.listdir(MTG_MANAGER))))
dshield_files = list(map(dshield_path, filter(lambda f: f.endswith(".csv"), os.listdir(MTG_DRAGONSHIELD))))

In [5]:
mtg_manager_collection = [Collection(f, source="MTGManager") for f in manager_files]
mtg_dshield_collection = [Collection(f, source="MTGDragonShield") for f in dshield_files]

In [6]:
collection_pairs = [(c1, c2) for c1, c2 in zip(mtg_manager_collection, mtg_dshield_collection)]

In [7]:
for i, pair in enumerate(collection_pairs):
    p1, p2 = pair
    print(f"{i+1:02d}) {p1.name} <- -> {p2.name}")

01) MTGManager/Black Collection <- -> MTGDragonShield/Black Collection
02) MTGManager/Non basic Lands <- -> MTGDragonShield/Non basic Lands
03) MTGManager/Artifacts <- -> MTGDragonShield/Artifacts
04) MTGManager/Green Collection <- -> MTGDragonShield/Green Collection
05) MTGManager/Islands <- -> MTGDragonShield/Islands
06) MTGManager/Gold Multicolor <- -> MTGDragonShield/Gold Multicolor
07) MTGManager/Mountains <- -> MTGDragonShield/Mountains
08) MTGManager/Swamps <- -> MTGDragonShield/Swamps
09) MTGManager/White Collection <- -> MTGDragonShield/White Collection
10) MTGManager/Blue Collection <- -> MTGDragonShield/Blue Collection
11) MTGManager/Red Collection <- -> MTGDragonShield/Red Collection
12) MTGManager/Forests <- -> MTGDragonShield/Forests
13) MTGManager/Plains <- -> MTGDragonShield/Plains


**Loading Original Data (from Apps) for reference**

In [8]:
orig_manager_path = partial(fullpath, base=MTG_MANAGER / "../original")
orig_dshield_path = partial(fullpath, base=MTG_DRAGONSHIELD / "../original")

orig_manager_files = list(map(orig_manager_path, filter(lambda f: f.endswith(".csv"), 
                                                        os.listdir(MTG_MANAGER / "../original"))))
orig_dshield_files = list(map(orig_dshield_path, filter(lambda f: f.endswith(".csv"), 
                                                        os.listdir(MTG_DRAGONSHIELD / "../original"))))

In [9]:
orig_manager_collection = [Collection(f, source="MTGManager") for f in orig_manager_files]
orig_dshield_collection = [Collection(f, source="MTGDragonShield") for f in orig_dshield_files]

**Load `Scryfall` Oracle**:

In [10]:
CARDS_ORACLE = ScryfallDB()

Loading Full Database file
Cards: 314861 [00:04, 66431.04/s] 

Loading Cards from Database into Oracle


### 3.1 Check length of Collections in pairs

First off, let's see whether the number of cards is the same between each pair

In [11]:
for i, pair in enumerate(collection_pairs):
    p1, p2 = pair
    print(f"Entries in {p1.source}/{p1.label}: {len(p1)}")
    print(f"Entries in {p2.source}/{p2.label}: {len(p2)}")
    same_count = "✅" if len(p1) == len(p2) else "❌"
    print(f"{same_count}")
    print("="*20)

Entries in MTGManager/Black Collection: 597
Entries in MTGDragonShield/Black Collection: 597
✅
Entries in MTGManager/Non basic Lands: 1580
Entries in MTGDragonShield/Non basic Lands: 1580
✅
Entries in MTGManager/Artifacts: 3571
Entries in MTGDragonShield/Artifacts: 3571
✅
Entries in MTGManager/Green Collection: 682
Entries in MTGDragonShield/Green Collection: 682
✅
Entries in MTGManager/Islands: 231
Entries in MTGDragonShield/Islands: 231
✅
Entries in MTGManager/Gold Multicolor: 1592
Entries in MTGDragonShield/Gold Multicolor: 1592
✅
Entries in MTGManager/Mountains: 416
Entries in MTGDragonShield/Mountains: 416
✅
Entries in MTGManager/Swamps: 248
Entries in MTGDragonShield/Swamps: 248
✅
Entries in MTGManager/White Collection: 4748
Entries in MTGDragonShield/White Collection: 4748
✅
Entries in MTGManager/Blue Collection: 5278
Entries in MTGDragonShield/Blue Collection: 5278
✅
Entries in MTGManager/Red Collection: 9426
Entries in MTGDragonShield/Red Collection: 9425
❌
Entries in MTGManag

⚠️

**All collections** in each pair has indeed _the same number of cards_ but **two**, namely _Red Collection_ and _Forests_.

This means we should expect some difference when comparing those. As for the others, any difference should now be in their <ins>very details</ins> [1](#fned). 

<span id="fned">**[1]**: I'd expect some due to _wrong_ matched `ExpansionCode` in step `2.3` (see [Collection Analysis](./Collection_Analysis.ipynb#2.3-Align-ExpansionCode) notebook), _ed._</span>

### 3.2 Comparing Collections

We are now _finally_ ready to proceed with the comparison of each pair, and to calculate their respective difference. On this note, let's keep in mind that substraction (as well as _difference_) is not commutative. Therefore, we will be applying the operation on both sides to spot differences in either direction!

In [12]:
from typing import Tuple 

def compare_collections(pair: Tuple[Collection, Collection]) -> None:
    p1, p2 = pair
    old_not_in_new = p1 - p2
    new_not_in_old = p2 - p1
    if not len(new_not_in_old) and not len(old_not_in_new):
        print(f'✅  Full Match between "{p1.name}" and "{p2.name}"')
    else:
        print(f'🔍  Diff between "{p1.name}" and "{p2.name}"')
        
        if len(new_not_in_old):
            print(f'\n\tEntries in "{p2.source}" not found in "{p1.source}".')
            display(new_not_in_old)
        if len(old_not_in_new):
            print(f'\n\tEntries in "{p1.source}" not found in "{p2.source}".')
            display(old_not_in_new)
    print("="*20, end="\n\n")

In [13]:
_ = list(map(compare_collections, collection_pairs))

✅  Full Match between "MTGManager/Black Collection" and "MTGDragonShield/Black Collection"

🔍  Diff between "MTGManager/Non basic Lands" and "MTGDragonShield/Non basic Lands"

	Entries in "MTGDragonShield" not found in "MTGManager".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
622,1,zoetic cavern,pg07,Gateway 2007,0.0,True,NearMint,English



	Entries in "MTGManager" not found in "MTGDragonShield".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
622,1,zoetic cavern,pgtw,Gateway 2006,0.0,True,NearMint,English



✅  Full Match between "MTGManager/Artifacts" and "MTGDragonShield/Artifacts"

🔍  Diff between "MTGManager/Green Collection" and "MTGDragonShield/Green Collection"

	Entries in "MTGDragonShield" not found in "MTGManager".


...


	Entries in "MTGManager" not found in "MTGDragonShield".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
508,2,devil's play,pmei,Magazine Inserts,0.0,True,NearMint,English
687,4,fated conflagration,pmei,Magazine Inserts,0.0,True,NearMint,English
688,2,fated conflagration,pmei,Magazine Inserts,0.0,True,NearMint,Italian
741,1,fireball,parl,Arena League 1996,0.0,False,NearMint,English
744,4,fireball,4ed,Fourth Edition,0.0,False,NearMint,English
933,1,genju of the spires,parl,Arena League 1996,0.0,False,NearMint,English
957,1,glacial ray,parl,Arena League 1996,0.0,False,NearMint,English
993,1,goblin dark-dwellers,pmei,Magazine Inserts,0.0,True,NearMint,English
1352,1,irencrag feat,peld,Throne of Eldraine Promos,0.0,False,NearMint,English
1713,1,nalathni dragon,pmei,Magazine Inserts,0.0,False,NearMint,English



🔍  Diff between "MTGManager/Forests" and "MTGDragonShield/Forests"

	Entries in "MTGManager" not found in "MTGDragonShield".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
82,1,forest,s00,Starter 2000,0.0,False,NearMint,Italian



✅  Full Match between "MTGManager/Plains" and "MTGDragonShield/Plains"



Let's now dig into each single difference to understand what's going on. 

**Target Collections to Examine**:

In particular we will inspect the following **`7`** Collections:

In [14]:
TARGET_INDICES = (1, 3, 5, 8, 9, 10, 11)
for i, idx in enumerate(TARGET_INDICES):
    print(f"{i+1:02d}) {collection_pairs[idx][0].label}")

01) Non basic Lands
02) Green Collection
03) Gold Multicolor
04) White Collection
05) Blue Collection
06) Red Collection
07) Forests


#### 3.2.1 `Non Basic Lands Collection`

**Report**

In [15]:
old, new = collection_pairs[1]
compare_collections((old, new))

🔍  Diff between "MTGManager/Non basic Lands" and "MTGDragonShield/Non basic Lands"

	Entries in "MTGDragonShield" not found in "MTGManager".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
622,1,zoetic cavern,pg07,Gateway 2007,0.0,True,NearMint,English



	Entries in "MTGManager" not found in "MTGDragonShield".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
622,1,zoetic cavern,pgtw,Gateway 2006,0.0,True,NearMint,English


In [16]:
old_orig = orig_manager_collection[1]
new_orig = orig_dshield_collection[1]

old_orig.name, new_orig.name

('MTGManager/Non basic Lands', 'MTGDragonShield/Non basic Lands')

In [17]:
old_orig[old_orig.Name == "Zoetic Cavern"]

,Quantity,Name,ExpansionCode,PurchasePrice,Foil,Condition,Language,PurchaseDate
622,1,Zoetic Cavern,pgtw,0.0,True,NearMint,English,2020-07-19


In [18]:
CARDS_ORACLE.expansion_codename_map["pgtw"]

'Gateway 2006'

In [19]:
CARDS_ORACLE.expansion_codename_map["pg07"]

'Gateway 2007'

In [20]:
list(CARDS_ORACLE.lookup("Zoetic Cavern", set_code="p"))

[Card(name='Zoetic Cavern', lang='en', set_code='pg07', set_name='Gateway 2007', set_type='promo')]

##### Response

- **MTGManager** (old) : ❌ 
- **MTGDragonShield** (new) : ✅ 

**Notes**: the inconsistency lies in _old_ **MTGManager** App data: `Zoetic Cavern` does **not** exist in `Gateway 2006` (`pgtw`), but it should be matched instead to `pg07` as correctly reported in _new_ **MTGDragonShield** App data.

---

#### 3.2.2 `Green Collection`

**Report:**

In [21]:
old, new = collection_pairs[3]
compare_collections((old, new))

🔍  Diff between "MTGManager/Green Collection" and "MTGDragonShield/Green Collection"

	Entries in "MTGDragonShield" not found in "MTGManager".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
92,1,elvish eulogist,dd1,Duel Decks: Elves vs. Goblins,0.07,False,NearMint,English



	Entries in "MTGManager" not found in "MTGDragonShield".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
90,1,elvish eulogist,evg,Duel Decks Anthology: Elves vs. Goblins,0.07,False,NearMint,English


In [22]:
old_orig = orig_manager_collection[3]
new_orig = orig_dshield_collection[3]

old_orig.name, new_orig.name

('MTGManager/Green Collection', 'MTGDragonShield/Green Collection')

In [23]:
list(CARDS_ORACLE.lookup("Elvish Eulogist"))

[Card(name='Elvish Eulogist', lang='en', set_code='dpa', set_name='Duels of the Planeswalkers', set_type='box'),
 Card(name='Elvish Eulogist', lang='en', set_code='dd1', set_name='Duel Decks: Elves vs. Goblins', set_type='duel_deck'),
 Card(name='Elvish Eulogist', lang='en', set_code='lrw', set_name='Lorwyn', set_type='expansion'),
 Card(name='Elvish Eulogist', lang='en', set_code='evg', set_name='Duel Decks Anthology: Elves vs. Goblins', set_type='duel_deck')]

##### Response

👔 **Tie** (_pun intended_)

- **MTGManager** (old) : ✅
- **MTGDragonShield** (new) : ✅ 

**Notes**: In this case both options seems correct, and indeed there is not even much difference between the two cards' art. 
In this case we might either a mismatch in data porting, or an error at the time of the initial (original) insert in the _old_ **MTGManager** App db. Therefore, I'm keen to consider the entry in **MTGDragonShield** App data correct after further investigation in my actual collection.

---

#### 3.2.3 `Gold and Multicolours Collection`

**Report:**

In [24]:
old, new = collection_pairs[5]
compare_collections((old, new))

🔍  Diff between "MTGManager/Gold Multicolor" and "MTGDragonShield/Gold Multicolor"

	Entries in "MTGDragonShield" not found in "MTGManager".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
333,1,hypersonic dragon,prtr,Return to Ravnica Promos,0.0,True,NearMint,English
575,1,rakshasa vizier,pktk,Khans of Tarkir Promos,0.0,True,NearMint,Italian
590,1,render silent,pdgm,Dragon's Maze Promos,0.0,True,NearMint,English
720,2,start // finish,akh,Amonkhet,0.0,False,NearMint,English



	Entries in "MTGManager" not found in "MTGDragonShield".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
239,1,start // finish,akh,Amonkhet,0.0,False,NearMint,English
334,1,hypersonic dragon,ppre,Prerelease Events,0.0,True,NearMint,English
576,1,rakshasa vizier,ppre,Prerelease Events,0.0,True,NearMint,Italian
591,1,render silent,pmei,Magazine Inserts,0.0,True,NearMint,English
721,1,start // finish,akh,Amonkhet,0.0,False,NearMint,English


In [25]:
old_orig = orig_manager_collection[5]
new_orig = orig_dshield_collection[5]

old_orig.name, new_orig.name

('MTGManager/Gold Multicolor', 'MTGDragonShield/Gold Multicolor')

##### CASE A. `Start // Finish`

Those two entries in the _old_ db look quite suspicious.

In [26]:
old_orig[old_orig.Name.str.startswith("Start")]

,Quantity,Name,ExpansionCode,PurchasePrice,Foil,Condition,Language,PurchaseDate
721,1,Start,akh,0.0,False,NearMint,English,2020-05-02
722,1,Start,akh,0.0,False,NearMint,Italian,2020-05-02


In [27]:
new_orig[new_orig.Name.str.startswith("Start")]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
722,2,Start // Finish,akh,Amonkhet,0.0,False,NearMint,English,2020-04-13
723,1,Start // Finish,akh,Amonkhet,0.0,False,NearMint,Italian,2020-02-05


In [28]:
old[old.Name == "Start // Finish"]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
239,1,Start // Finish,akh,Amonkhet,0.0,False,NearMint,English,2020-04-13
721,1,Start // Finish,akh,Amonkhet,0.0,False,NearMint,English,2020-05-02
722,1,Start // Finish,akh,Amonkhet,0.0,False,NearMint,Italian,2020-05-02


In [29]:
new[new.Name == "Start // Finish"]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
720,2,Start // Finish,akh,Amonkhet,0.0,False,NearMint,English,2020-04-13
721,1,Start // Finish,akh,Amonkhet,0.0,False,NearMint,Italian,2020-02-05



**Conclusion**: ⚠️ 

As expected, the inconsistency is _due_ to the **error** in the new app with `PurchaseDate` parsing. 

We have verified that the _diff_ is not due to any mistake in the previous _preparatory_ changes of the data. 
In fact, both **number of cards** and **expansion codes** are all correct, just the corresponding `Purchase Date` is inconsitent between the two App.

Not a big deal in general, but to be reported.

##### CASE B. `Hypersonic Dragon`

In [30]:
old_orig[old_orig.Name.str.startswith("Hypersonic Dragon")]

,Quantity,Name,ExpansionCode,PurchasePrice,Foil,Condition,Language,PurchaseDate
333,1,Hypersonic Dragon,gk1_izzet,0.0,False,NearMint,English,2020-04-13
334,1,Hypersonic Dragon,ppre,0.0,True,NearMint,English,2020-07-18


In [31]:
new_orig[new_orig.Name.str.startswith("Hypersonic Dragon")]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
333,1,Hypersonic Dragon,gk1_izzet,Guild Kit: Izzet,0.0,False,NearMint,English,2020-04-13
334,1,Hypersonic Dragon,prtr,Return to Ravnica Promos,0.0,True,NearMint,English,2020-07-18


In [32]:
old[old.Name == "Hypersonic Dragon"]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
333,1,Hypersonic Dragon,gk1_izzet,Guild Kit: Izzet,0.0,False,NearMint,English,2020-04-13
334,1,Hypersonic Dragon,ppre,Prerelease Events,0.0,True,NearMint,English,2020-07-18


In [33]:
new[new.Name == "Hypersonic Dragon"]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
332,1,Hypersonic Dragon,gk1_izzet,Guild Kit: Izzet,0.0,False,NearMint,English,2020-04-13
333,1,Hypersonic Dragon,prtr,Return to Ravnica Promos,0.0,True,NearMint,English,2020-07-18


In [34]:
list(CARDS_ORACLE.lookup("Hypersonic Dragon"))

[Card(name='Hypersonic Dragon', lang='en', set_code='ima', set_name='Iconic Masters', set_type='masters'),
 Card(name='Hypersonic Dragon', lang='en', set_code='gk1', set_name='GRN Guild Kit', set_type='box'),
 Card(name='Hypersonic Dragon', lang='en', set_code='prtr', set_name='Return to Ravnica Promos', set_type='promo'),
 Card(name='Hypersonic Dragon', lang='en', set_code='rtr', set_name='Return to Ravnica', set_type='expansion')]

**Conclucions**


- **MTGManager** (old) : ❌
- **MTGDragonShield** (new) : ✅ 


**Notes**: The inconsistency is in the _old_ App Database: `Hypersonic Dragon` in `ppre` does not exist. 
It does exist, on the other hand, in `prtr` as correctly reported in the _new_ App.

##### CASE C. `Render Silent`

In [35]:
list(CARDS_ORACLE.lookup("Render Silent"))

[Card(name='Render Silent', lang='en', set_code='pdgm', set_name="Dragon's Maze Promos", set_type='promo'),
 Card(name='Render Silent', lang='en', set_code='gk2', set_name='RNA Guild Kit', set_type='box'),
 Card(name='Render Silent', lang='en', set_code='dgm', set_name="Dragon's Maze", set_type='expansion')]

In [36]:
old[old.Name == "Render Silent"]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
591,1,Render Silent,pmei,Magazine Inserts,0.0,True,NearMint,English,2020-04-02


In [37]:
new[new.Name == "Render Silent"]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
590,1,Render Silent,pdgm,Dragon's Maze Promos,0.0,True,NearMint,English,2020-02-04


**Conclucions**

- **MTGManager** (old) : ❌
- **MTGDragonShield** (new) : ✅ 

**Notes**: Same case as for the `Hypersonic Dragon`. The inconsistency lies in the _old_ App Database:
`Render Silent` in `pmei` does not exist, whereas it does exist in `pdgm` as correctly reported in _new_ App.

##### CASE D. `Rakshasa Vizier`

In [38]:
list(CARDS_ORACLE.lookup("rakshasa vizier"))

[Card(name='Rakshasa Vizier', lang='en', set_code='pktk', set_name='Khans of Tarkir Promos', set_type='promo'),
 Card(name='Rakshasa Vizier', lang='en', set_code='ktk', set_name='Khans of Tarkir', set_type='expansion'),
 Card(name='Rakshasa Vizier', lang='en', set_code='pktk', set_name='Khans of Tarkir Promos', set_type='promo')]

In [39]:
old[old.Name == "Rakshasa Vizier"]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
574,2,Rakshasa Vizier,ktk,Khans of Tarkir,0.0,False,NearMint,English,2020-05-02
575,1,Rakshasa Vizier,ktk,Khans of Tarkir,0.0,False,NearMint,Italian,2020-05-02
576,1,Rakshasa Vizier,ppre,Prerelease Events,0.0,True,NearMint,Italian,2020-04-14


In [40]:
new[new.Name == "Rakshasa Vizier"]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
573,2,Rakshasa Vizier,ktk,Khans of Tarkir,0.0,False,NearMint,English,2020-02-05
574,1,Rakshasa Vizier,ktk,Khans of Tarkir,0.0,False,NearMint,Italian,2020-02-05
575,1,Rakshasa Vizier,pktk,Khans of Tarkir Promos,0.0,True,NearMint,Italian,2020-04-14


**Conclusions**

- **MTGManager** (old) : ❌
- **MTGDragonShield** (new) : ✅ 


**Notes**: Another inconsistency in matched expansion set. 

##### Response

- **MTGManager** (old) : ❌ + ⚠️
- **MTGDragonShield** (new) : ✅  + ⚠️

**Notes**: Four (out of 5) differences occur because of incorrect matches to the `ExpansionCode` in the _old_ App that has been _fixed_ in the new App. The fifth case is due to error in _date_ parsing across the two Apps, producing wrong aggregated entries in the _new_ App Collection.


---

#### 3.2.4 `White Collection`

**Report:**

In [41]:
old, new = collection_pairs[8]
compare_collections((old, new))

🔍  Diff between "MTGManager/White Collection" and "MTGDragonShield/White Collection"

	Entries in "MTGDragonShield" not found in "MTGManager".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
312,1,calciderm,pg07,Gateway 2007,0.0,True,NearMint,English



	Entries in "MTGManager" not found in "MTGDragonShield".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
309,1,calciderm,pgtw,Gateway 2006,0.0,True,NearMint,English


In [42]:
list(CARDS_ORACLE.lookup("Calciderm"))

[Card(name='Calciderm', lang='en', set_code='ema', set_name='Eternal Masters', set_type='masters'),
 Card(name='Calciderm', lang='en', set_code='pg07', set_name='Gateway 2007', set_type='promo'),
 Card(name='Calciderm', lang='en', set_code='tsr', set_name='Time Spiral Remastered', set_type='masters'),
 Card(name='Calciderm', lang='en', set_code='plc', set_name='Planar Chaos', set_type='expansion')]

##### Response

- **MTGManager** (old) : ❌ 
- **MTGDragonShield** (new) : ✅ 

**Notes**: Expansion `pg07` reported in  _new_ App is correct for `Calciderm`. 

---

#### 3.2.5 `Blue Collection`

**Report:**

In [43]:
old, new = collection_pairs[9]
compare_collections((old, new))

🔍  Diff between "MTGManager/Blue Collection" and "MTGDragonShield/Blue Collection"

	Entries in "MTGDragonShield" not found in "MTGManager".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
771,2,soul seizer // ghastly haunting,dka,Dark Ascension,0.0,False,NearMint,Italian
1626,1,scourge of fleets,pjou,Journey into Nyx Promos,0.0,True,NearMint,English
2058,1,trail of mystery,pktk,Khans of Tarkir Promos,0.0,False,NearMint,English



	Entries in "MTGManager" not found in "MTGDragonShield".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
769,1,soul seizer // ghastly haunting,dka,Dark Ascension,0.0,False,NearMint,Italian
1622,1,scourge of fleets,ppre,Prerelease Events,0.0,True,NearMint,English
1820,1,soul seizer // ghastly haunting,dka,Dark Ascension,0.0,False,NearMint,Italian
2054,1,trail of mystery,ppre,Prerelease Events,0.0,False,NearMint,English


In [44]:
list(CARDS_ORACLE.lookup("Scourge of Fleets"))

[Card(name='Scourge of Fleets', lang='en', set_code='pjou', set_name='Journey into Nyx Promos', set_type='promo'),
 Card(name='Scourge of Fleets', lang='en', set_code='jou', set_name='Journey into Nyx', set_type='expansion'),
 Card(name='Scourge of Fleets', lang='en', set_code='ddo', set_name='Duel Decks: Elspeth vs. Kiora', set_type='duel_deck'),
 Card(name='Scourge of Fleets', lang='en', set_code='cmr', set_name='Commander Legends', set_type='draft_innovation'),
 Card(name='Scourge of Fleets', lang='en', set_code='znc', set_name='Zendikar Rising Commander', set_type='commander')]

In [45]:
list(CARDS_ORACLE.lookup("Trail of Mystery"))

[Card(name='Trail of Mystery', lang='en', set_code='c19', set_name='Commander 2019', set_type='commander'),
 Card(name='Trail of Mystery', lang='en', set_code='pktk', set_name='Khans of Tarkir Promos', set_type='promo'),
 Card(name='Trail of Mystery', lang='en', set_code='ktk', set_name='Khans of Tarkir', set_type='expansion')]

##### Response

- **MTGManager** (old) : ❌ + ⚠️
- **MTGDragonShield** (new) : ✅  + ⚠️

**Notes**: Situation very similar to `Gold and Multicolours` Collection:

1. `ExpansionCode` mismatch due to incorrect mapping in the _old_ App that, correctly _fixed_ in the new App. 
2. One duplicated entry due to errors in _date_ parsing across the two Apps

---

#### 3.2.6 `Red Collection`

**Report:**

In [46]:
old, new = collection_pairs[10]
compare_collections((old, new))

🔍  Diff between "MTGManager/Red Collection" and "MTGDragonShield/Red Collection"

	Entries in "MTGDragonShield" not found in "MTGManager".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
809,2,devil's play,pisd,Innistrad Promos,0.0,True,NearMint,English
1085,4,fated conflagration,pbng,Born of the Gods Promos,0.0,True,NearMint,English
1086,2,fated conflagration,pbng,Born of the Gods Promos,0.0,True,NearMint,Italian
1176,5,fireball,4ed,Fourth Edition,0.0,False,NearMint,English
1453,1,genju of the spires,pal05,Arena League 2005,0.0,False,NearMint,English
1488,1,glacial ray,pal04,Arena League 2004,0.0,False,NearMint,English
1542,1,goblin dark-dwellers,pogw,Oath of the Gatewatch Promos,0.0,True,NearMint,English
2066,1,irencrag feat,veld,Throne of Eldraine Variants,0.0,False,NearMint,English
3401,4,slaying fire,peld,Throne of Eldraine Promos,0.0,False,NearMint,English
4062,1,windseeker centaur,phpr,HarperPrism Book Promos,0.0,False,NearMint,English



	Entries in "MTGManager" not found in "MTGDragonShield".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
508,2,devil's play,pmei,Magazine Inserts,0.0,True,NearMint,English
687,4,fated conflagration,pmei,Magazine Inserts,0.0,True,NearMint,English
688,2,fated conflagration,pmei,Magazine Inserts,0.0,True,NearMint,Italian
741,1,fireball,parl,Arena League 1996,0.0,False,NearMint,English
744,4,fireball,4ed,Fourth Edition,0.0,False,NearMint,English
933,1,genju of the spires,parl,Arena League 1996,0.0,False,NearMint,English
957,1,glacial ray,parl,Arena League 1996,0.0,False,NearMint,English
993,1,goblin dark-dwellers,pmei,Magazine Inserts,0.0,True,NearMint,English
1352,1,irencrag feat,peld,Throne of Eldraine Promos,0.0,False,NearMint,English
1713,1,nalathni dragon,pmei,Magazine Inserts,0.0,False,NearMint,English


In [47]:
old_orig = orig_manager_collection[10]
new_orig = orig_dshield_collection[10]

old_orig.name, new_orig.name

('MTGManager/Red Collection', 'MTGDragonShield/Red Collection')

##### CASE A, B: `Devil's Play`, `Fated Conflagration`

In [48]:
list(CARDS_ORACLE.lookup("Devil's Play"))

[Card(name="Devil's Play", lang='en', set_code='ddk', set_name='Duel Decks: Sorin vs. Tibalt', set_type='duel_deck'),
 Card(name="Devil's Play", lang='en', set_code='isd', set_name='Innistrad', set_type='expansion'),
 Card(name="Devil's Play", lang='en', set_code='c19', set_name='Commander 2019', set_type='commander'),
 Card(name="Devil's Play", lang='en', set_code='pisd', set_name='Innistrad Promos', set_type='promo'),
 Card(name="Devil's Play", lang='en', set_code='ha3', set_name='Historic Anthology 3', set_type='box')]

In [49]:
list(CARDS_ORACLE.lookup("Fated Conflagration"))

[Card(name='Fated Conflagration', lang='en', set_code='bng', set_name='Born of the Gods', set_type='expansion'),
 Card(name='Fated Conflagration', lang='en', set_code='pbng', set_name='Born of the Gods Promos', set_type='promo')]

**Conclusions**

- **MTGManager** (old) : ❌
- **MTGDragonShield** (new) : ✅ 

**Notes**: Neiter `Devil's Play` nor `Fated Conflagration` appear in the Oracle as `Magazine Inserts` sets. This has been already _fixed_ in the _new_ App. **Besides**, the number of cards in the two collection sets correspond.


##### CASE C, D: `Genju of the Spires`, `Glacial Ray`

In [50]:
list(CARDS_ORACLE.lookup("Genju of the Spires"))

[Card(name='Genju of the Spires', lang='en', set_code='mb1', set_name='Mystery Booster', set_type='masters'),
 Card(name='Genju of the Spires', lang='en', set_code='pal05', set_name='Arena League 2005', set_type='promo'),
 Card(name='Genju of the Spires', lang='en', set_code='a25', set_name='Masters 25', set_type='masters'),
 Card(name='Genju of the Spires', lang='en', set_code='bok', set_name='Betrayers of Kamigawa', set_type='expansion')]

In [51]:
list(CARDS_ORACLE.lookup("Glacial Ray"))

[Card(name='Glacial Ray', lang='en', set_code='pal04', set_name='Arena League 2004', set_type='promo'),
 Card(name='Glacial Ray', lang='en', set_code='chk', set_name='Champions of Kamigawa', set_type='expansion'),
 Card(name='Glacial Ray', lang='en', set_code='mma', set_name='Modern Masters', set_type='masters')]

In [52]:
old_orig[old_orig.Name.isin(["Genju of the Spires", "Glacial Ray"])]

,Quantity,Name,ExpansionCode,PurchasePrice,Foil,Condition,Language,PurchaseDate
932,1,Genju of the Spires,a25,0.0,False,NearMint,English,2019-03-24
933,1,Genju of the Spires,parl,0.0,False,NearMint,English,2020-04-19
957,1,Glacial Ray,parl,0.0,False,NearMint,English,2020-04-19


In [53]:
CARDS_ORACLE.expansion_codename_map["parl"]

'Arena League 1996'

In [54]:
CARDS_ORACLE.expansion_codename_map["pmei"]

'Magazine Inserts'

**Conclusions**

- **MTGManager** (old) : ❌
- **MTGDragonShield** (new) : ✅ 

**Notes**: Entries in the _old_ data for both `Glacial Ray` and `Genju of the Spires` are incorrectly matched to `parl` (i.e. _Arena League 1996_), whereas the two cards in the _new_ App  have been correctly re-mapped to _Arena League 2004_ and _Arena League 2005_, respectively.

##### Case E: `Windseeker Centaur`

In [55]:
list(CARDS_ORACLE.lookup("Windseeker centaur"))

[Card(name='Windseeker Centaur', lang='en', set_code='phpr', set_name='HarperPrism Book Promos', set_type='promo')]

**Conclusions**

- **MTGManager** (old) : ❌
- **MTGDragonShield** (new) : ✅ 

**Notes**: This was easy to resolve, and **does not** require any intervention as the collection in the _new_ App correctly remapped the cards from `pmei` to `phpr`, as also reported in Scryfall Oracle.

##### CASE F: `Fireball`

In [56]:
fireball_old = old[old.Name.str.lower() == "fireball"]
fireball_old

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
741,1,Fireball,parl,Arena League 1996,0.0,False,NearMint,English,2020-04-19
742,1,Fireball,pd2,Premium Deck Series: Fire and Lightning,0.0,True,NearMint,English,2019-03-03
743,1,Fireball,pmei,Magazine Inserts,0.0,False,NearMint,English,2020-08-05
744,4,Fireball,4ed,Fourth Edition,0.0,False,NearMint,English,2020-03-05
745,4,Fireball,4ed,Fourth Edition,0.0,True,NearMint,Italian,2019-02-24
746,1,Fireball,3ed,Revised Edition,0.0,False,NearMint,English,2018-07-28
747,1,Fireball,3ed,Revised Edition,0.0,False,Good,English,2020-08-05
748,2,Fireball,3ed,Revised Edition,0.0,True,NearMint,Spanish,2020-08-05
749,1,Fireball,p05,Magic Player Rewards 2005,0.0,False,NearMint,English,2020-08-05
750,1,Fireball,2ed,Unlimited Edition,0.0,False,NearMint,English,2019-01-03


In [57]:
fireball_old.Quantity.sum()

30

In [58]:
fireball_new = new[new.Name == "Fireball"]
fireball_new

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
1173,1,Fireball,p05,Magic Player Rewards 2005,0.0,False,NearMint,English,2020-04-19
1174,1,Fireball,pd2,Premium Deck Series: Fire and Lightning,0.0,True,NearMint,English,2019-03-03
1175,1,Fireball,pmei,Magazine Inserts,0.0,False,NearMint,English,2020-05-08
1176,5,Fireball,4ed,Fourth Edition,0.0,False,NearMint,English,2020-05-03
1177,4,Fireball,4ed,Fourth Edition,0.0,True,NearMint,Italian,2019-02-24
1178,2,Fireball,4ed,Fourth Edition,0.0,True,NearMint,Spanish,2019-02-24
1179,1,Fireball,3ed,Revised Edition,0.0,False,NearMint,English,2018-07-28
1180,1,Fireball,3ed,Revised Edition,0.0,False,Good,English,2020-05-08
1181,5,Fireball,3ed,Revised Edition,0.0,True,NearMint,Italian,2019-02-23
1182,2,Fireball,3ed,Revised Edition,0.0,True,NearMint,Spanish,2020-05-08


In [59]:
fireball_new.Quantity.sum()

30

In [60]:
on_diff = (old - new)
on_diff[on_diff.Name == "fireball"]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
741,1,fireball,parl,Arena League 1996,0.0,False,NearMint,English
744,4,fireball,4ed,Fourth Edition,0.0,False,NearMint,English


In [61]:
no_diff = (new - old)
no_diff[no_diff.Name == "fireball"]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
1176,5,fireball,4ed,Fourth Edition,0.0,False,NearMint,English


In [62]:
list(CARDS_ORACLE.lookup("fireball", set_type="promo"))

[Card(name='Fireball', lang='en', set_code='parl', set_name='Arena League 1996', set_type='promo'),
 Card(name='Fireball', lang='en', set_code='pmei', set_name='Magazine Inserts', set_type='promo'),
 Card(name='Fireball', lang='en', set_code='p05', set_name='Magic Player Rewards 2005', set_type='promo')]

In [63]:
old_orig[((old_orig.Name == "Fireball") & (old_orig.ExpansionCode.str.startswith("p")))]

,Quantity,Name,ExpansionCode,PurchasePrice,Foil,Condition,Language,PurchaseDate
741,1,Fireball,pmpr,0.0,False,NearMint,English,2020-04-19
742,1,Fireball,pd2,0.0,True,NearMint,English,2019-03-03
743,1,Fireball,pmei,0.0,False,NearMint,English,2020-08-05


In [64]:
"pmpr" in CARDS_ORACLE.expansion_codename_map

False

**Conclusions**

- **MTGManager** (old) : ⚠️
- **MTGDragonShield** (new) : ⚠️ + ❌

**Notes**: **Two** issues to report here. First off, in the _old_ app, some entries have been saved in _lowercase_, which could have produced _false_ missing entries. Fortunately this has been resolved by not accounting for the case in card names when applying the comparison.

The second (and more relevant) issue is in a single entry of `Fireball` mistakenly mapped to an non-existing `pmpr` Expansion set. This set has been remapped to the most _plausible_ PROMO one in this notebook, i.e. `parl` _Arena League 1996_. In the _new_ app though this has turned into an extra `4ED` card, and so the mismatch.

📝 **TODO**: To be fixed manually in the **MTG DragonShield** App.


##### CASE G, H: `Irencrag Feat` and `Slaying fire`

In [65]:
old[old.Name.isin(["Irencrag Feat", "Slaying Fire"])]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
1352,1,Irencrag Feat,peld,Throne of Eldraine Promos,0.0,False,NearMint,English,2020-03-22
1353,4,Irencrag Feat,eld,Throne of Eldraine,0.0,False,NearMint,English,2020-03-22
2264,4,Slaying Fire,eld,Throne of Eldraine,0.0,False,NearMint,English,2020-03-22


In [66]:
new[new.Name.isin(["Irencrag Feat", "Slaying Fire"])]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
2066,1,Irencrag Feat,veld,Throne of Eldraine Variants,0.0,False,NearMint,English,2020-03-22
2067,4,Irencrag Feat,eld,Throne of Eldraine,0.0,False,NearMint,English,2020-03-22
3401,4,Slaying Fire,peld,Throne of Eldraine Promos,0.0,False,NearMint,English,2020-03-22


In [67]:
list(CARDS_ORACLE.lookup("Irencrag Feat"))

[Card(name='Irencrag Feat', lang='en', set_code='eld', set_name='Throne of Eldraine', set_type='expansion'),
 Card(name='Irencrag Feat', lang='en', set_code='peld', set_name='Throne of Eldraine Promos', set_type='promo'),
 Card(name='Irencrag Feat', lang='en', set_code='peld', set_name='Throne of Eldraine Promos', set_type='promo'),
 Card(name='Irencrag Feat', lang='en', set_code='eld', set_name='Throne of Eldraine', set_type='expansion')]

In [68]:
list(CARDS_ORACLE.lookup("Slaying Fire"))

[Card(name='Slaying Fire', lang='en', set_code='eld', set_name='Throne of Eldraine', set_type='expansion'),
 Card(name='Slaying Fire', lang='en', set_code='eld', set_name='Throne of Eldraine', set_type='expansion')]

In [69]:
old_orig[old_orig.Name.isin(["Irencrag Feat", "Slaying Fire"])]

,Quantity,Name,ExpansionCode,PurchasePrice,Foil,Condition,Language,PurchaseDate
1353,1,Irencrag Feat,peld,0.0,False,NearMint,English,2020-03-22
1354,4,Irencrag Feat,eld,0.0,False,NearMint,English,2020-03-22
2265,4,Slaying Fire,eld,0.0,False,NearMint,English,2020-03-22


**Conclusions**

- **MTGManager** (old) : ✅
- **MTGDragonShield** (new) : ❌

**Notes**: This is the **first** time that actually data in the _old_ db are indeed correct wrt. the _new_ one. In particular: 

📝 **TODO**: To be fixed manually in the **MTG DragonShield** App.

- `Irencrag Feat` should be mapped to `peld` (if available in App)
- `Slaying Fire` should be mapped to `eld` (checked also w/ physical cards and I do not have promo!)




##### CASE J: `Goblin Dark-Dwellers`

In [70]:
list(CARDS_ORACLE.lookup("Goblin Dark-Dwellers"))

[Card(name='Goblin Dark-Dwellers', lang='en', set_code='c20', set_name='Commander 2020', set_type='commander'),
 Card(name='Goblin Dark-Dwellers', lang='en', set_code='pogw', set_name='Oath of the Gatewatch Promos', set_type='promo'),
 Card(name='Goblin Dark-Dwellers', lang='en', set_code='ogw', set_name='Oath of the Gatewatch', set_type='expansion'),
 Card(name='Goblin Dark-Dwellers', lang='en', set_code='pogw', set_name='Oath of the Gatewatch Promos', set_type='promo')]

**Conclusions**

- **MTGManager** (old) : ❌
- **MTGDragonShield** (new) : ✅

**Notes**: The mismatch is due to inconsistency in the _old_ app data, correctly FIXED in the _new_ APP: from `pmei` to `pogw`

##### CASE K: `Nalathni Dragon`

In [71]:
list(CARDS_ORACLE.lookup("Nalathni Dragon"))

[Card(name='Nalathni Dragon', lang='en', set_code='pdrc', set_name='Dragon Con', set_type='promo')]

**Conclusions**

- **MTGManager** (old) : ✅
- **MTGDragonShield** (new) : ❌

**Notes**: This card is actually missing in the _new_ collection and the **one** we were looking for to understand numbers mismatch!

📝 **TODO**: To be fixed manually in the **MTG DragonShield** App.

- `Nalathni Dragon` should be mapped to `pdrc` (if available in App)



##### Response

**Conclusions**

- **MTGManager** (old) : ⚠️ (but also ✅ )
- **MTGDragonShield** (new) : 📝 + ⚠️ 

**Notes**: Generally data in the _new_ App are correct, but differently from the other cases analysed, this time collection requires **manual interventions**:

- one `Fireball` entry from `4ed` should be moved to `Arena League 1996` (if available in App)
- `Irencrag Feat` should be mapped to `peld` (promo, if available in App)
- `Slaying Fire` should be mapped to `eld` (no promo!)
- `Nalathni Dragon` is missing and should be added!

---

#### 3.2.7 `Forests Lands Collection`

**Report:**

In [72]:
old, new = collection_pairs[11]
compare_collections((old, new))

🔍  Diff between "MTGManager/Forests" and "MTGDragonShield/Forests"

	Entries in "MTGManager" not found in "MTGDragonShield".


,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language
82,1,forest,s00,Starter 2000,0.0,False,NearMint,Italian


In [73]:
list(CARDS_ORACLE.lookup("forest", set_code="s"))

[Card(name='Forest', lang='en', set_code='sld', set_name='Secret Lair Drop', set_type='box'),
 Card(name='Forest', lang='en', set_code='soi', set_name='Shadows over Innistrad', set_type='expansion'),
 Card(name='Forest', lang='en', set_code='sld', set_name='Secret Lair Drop', set_type='box'),
 Card(name='Forest', lang='en', set_code='s99', set_name='Starter 1999', set_type='starter'),
 Card(name='Forest', lang='en', set_code='som', set_name='Scars of Mirrodin', set_type='expansion'),
 Card(name='Forest', lang='en', set_code='soi', set_name='Shadows over Innistrad', set_type='expansion'),
 Card(name='Forest', lang='en', set_code='sum', set_name='Summer Magic / Edgar', set_type='core'),
 Card(name='Forest', lang='en', set_code='som', set_name='Scars of Mirrodin', set_type='expansion'),
 Card(name='Forest', lang='en', set_code='shm', set_name='Shadowmoor', set_type='expansion'),
 Card(name='Forest', lang='en', set_code='shm', set_name='Shadowmoor', set_type='expansion'),
 Card(name='Fores

##### Response

**Conclusions**

- **MTGManager** (old) : ❌
- **MTGDragonShield** (new) : 📝

**Notes**: This card is actually missing in the _new_ collection and the **one** we were looking for to finally find out **all** the missing ones. 

📝 **TODO**: To be fixed manually in the **MTG DragonShield** App.

- `Forest` should be mapped to `s99` (if available in App) and not `s00` as mistakenly reported in _old_ App.

